In [ ]:
%cd ~/eda-repo-3

/home/park/eda-repo-3


In [3]:
# 펫 건강 케어 추천

import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64

from dotenv import load_dotenv
import os
import json

# .env 파일을 찾아 환경 변수로 로드
load_dotenv()

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

# ▶️ 반려동물 등록 데이터 가져오기
query = """
SELECT *
FROM companion_animal_registration
"""
df = pd.read_sql(query, engine)

# ▶️ 행정구역 명칭을 folium key와 맞추기 위해 하나로 합치기
df['region'] = df['sigungu']

# 'pet_care'행 추가
pet_care_map = {
    '강북구': 1,
    '은평구': 1,
    '서대문구': 1,
}


df['pet_care'] = df['sigungu'].map(pet_care_map)

# 색깔 열 추가
# pet_care가 있으면 주황색, 없으면 노랑색 지정
def color_map(pet_care):
    if pd.isna(pet_care):
        return 'yellow'
    else:
        return 'orange'

df['color'] = df['pet_care'].apply(color_map)





# ▶️ GeoJSON 경로 지정
geo_path = '../../../DATA/02. skorea_municipalities_geo_simple.json'  # 반드시 'region' 컬럼과 feature.properties.name이 일치해야 함

# ▶️ GeoJSON 로드 (시도+시군구 이름이 properties.name으로 되어 있어야 함)
with open(geo_path, 'r', encoding='utf-8') as f:
    geo = json.load(f)

# ▶️ 지도 시각화
m = folium.Map(location=[36.5, 127.5], zoom_start=8)

# 3. GeoJSON에서 각 구의 중심좌표(centroid) 추출
regions, lats, lngs = [], [], []
for feature in geo['features']:
    region_name = feature['properties']['name']
    geometry = feature['geometry']
    centroid = shape(geometry).centroid
    regions.append(region_name)
    lats.append(centroid.y)
    lngs.append(centroid.x)
geo_df = pd.DataFrame({'region': regions, 'lat': lats, 'lng': lngs})

# 4. 기존 데이터와 중심좌표 병합
df = pd.merge(df, geo_df, on='region', how='left')

# 스타일 함수 정의
def style_function(feature):
    region_name = feature['properties']['name']
    color = df.loc[df['region'] == region_name, 'color']
    if not color.empty:
        return {'fillColor': color.values[0], 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

# GeoJson 레이어 추가
folium.GeoJson(
    geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'])
).add_to(m)

# 마커 스타일 개선 (더 눈에 잘 보이도록)
for idx, row in df.iterrows():
    if not pd.isna(row['pet_care']):
        folium.Marker(
            location=[row['lat'], row['lng']],
            icon=folium.DivIcon(
                html=f"""<div style="
                    font-size: 13px; 
                    color: white; 
                    background: rgba(30, 144, 255, 0.8); 
                    border-radius: 15px; 
                    padding: 6px 10px;
                    font-weight: bold;
                    text-align: center;
                    border: 2px solid white;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.3);
                    white-space: nowrap;
                    min-width: 150px;
                ">펫 건강 케어 : {row['region']}</div>""",
                icon_size=(80, 30),
                icon_anchor=(40, 15)
            ),
            tooltip=f"{row['region']} 순위: {int(row['pet_care'])}"
        ).add_to(m)

# 8. 지도 저장 또는 출력
m.save('../../../RESULT/visualization/pet_health_care_folium_choropleth.html')
m

In [ ]:
# 펫 스트레스-프리

import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64

from dotenv import load_dotenv
import os
import json

# .env 파일을 찾아 환경 변수로 로드
load_dotenv()

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

# ▶️ 반려동물 등록 데이터 가져오기
query = """
SELECT *
FROM companion_animal_registration
"""
df = pd.read_sql(query, engine)

# ▶️ 행정구역 명칭을 folium key와 맞추기 위해 하나로 합치기
df['region'] = df['sigungu']

# 'pet_care'행 추가
pet_care_map = {
    '강남구': 1,
    '서초구': 1,
    '송파구': 1,
}


df['pet_care'] = df['sigungu'].map(pet_care_map)

# 색깔 열 추가
# pet_care가 있으면 주황색, 없으면 노랑색 지정
def color_map(pet_care):
    if pd.isna(pet_care):
        return 'yellow'
    else:
        return 'orange'

df['color'] = df['pet_care'].apply(color_map)





# ▶️ GeoJSON 경로 지정
geo_path = 'DATA/02. skorea_municipalities_geo_simple.json'  # 반드시 'region' 컬럼과 feature.properties.name이 일치해야 함

# ▶️ GeoJSON 로드 (시도+시군구 이름이 properties.name으로 되어 있어야 함)
with open(geo_path, 'r', encoding='utf-8') as f:
    geo = json.load(f)

# ▶️ 지도 시각화
m = folium.Map(location=[36.5, 127.5], zoom_start=8)

# 3. GeoJSON에서 각 구의 중심좌표(centroid) 추출
regions, lats, lngs = [], [], []
for feature in geo['features']:
    region_name = feature['properties']['name']
    geometry = feature['geometry']
    centroid = shape(geometry).centroid
    regions.append(region_name)
    lats.append(centroid.y)
    lngs.append(centroid.x)
geo_df = pd.DataFrame({'region': regions, 'lat': lats, 'lng': lngs})

# 4. 기존 데이터와 중심좌표 병합
df = pd.merge(df, geo_df, on='region', how='left')

# 스타일 함수 정의
def style_function(feature):
    region_name = feature['properties']['name']
    color = df.loc[df['region'] == region_name, 'color']
    if not color.empty:
        return {'fillColor': color.values[0], 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

# GeoJson 레이어 추가
folium.GeoJson(
    geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'])
).add_to(m)

# 마커 스타일 개선 (더 눈에 잘 보이도록)
for idx, row in df.iterrows():
    if not pd.isna(row['pet_care']):
        folium.Marker(
            location=[row['lat'], row['lng']],
            icon=folium.DivIcon(
                html=f"""<div style="
                    font-size: 13px; 
                    color: white; 
                    background: rgba(30, 144, 255, 0.8); 
                    border-radius: 15px; 
                    padding: 6px 10px;
                    font-weight: bold;
                    text-align: center;
                    border: 2px solid white;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.3);
                    white-space: nowrap;
                    min-width: 170px;
                ">펫 스트레스-프리 : {row['region']}</div>""",
                icon_size=(80, 30),
                icon_anchor=(40, 15)
            ),
            tooltip=f"{row['region']}"
        ).add_to(m)

# 8. 지도 저장 또는 출력
m.save('RESULT/visualization/pet_stress_free_folium_choropleth.html')
m

In [9]:
# 펫 산책, 놀이 중심

import pandas as pd
import matplotlib.pyplot as plt
import koreanize_matplotlib
import folium
from sqlalchemy import create_engine
from shapely.geometry import shape
from io import BytesIO
import base64

from dotenv import load_dotenv
import os
import json

# .env 파일을 찾아 환경 변수로 로드
load_dotenv()

# 환경 변수 가져오기
host = os.getenv("DB_HOST")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
database = os.getenv("DB_NAME")

# ▶️ MySQL 연결
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:3306/{database}")

# ▶️ 반려동물 등록 데이터 가져오기
query = """
SELECT *
FROM companion_animal_registration
"""
df = pd.read_sql(query, engine)

# ▶️ 행정구역 명칭을 folium key와 맞추기 위해 하나로 합치기
df['region'] = df['sigungu']

# 'pet_play_map'행 추가
pet_play_map = {
    '종로구': 1,
    '중구': 1,
    '서초구': 1,
}


df['pet_play'] = df['sigungu'].map(pet_play_map)

# 색깔 열 추가
# pet_play 있으면 주황색, 없으면 노랑색 지정
def color_map(pet_play):
    if pd.isna(pet_play):
        return 'yellow'
    else:
        return 'orange'

df['color'] = df['pet_play'].apply(color_map)





# ▶️ GeoJSON 경로 지정
geo_path = '../../../DATA/02. skorea_municipalities_geo_simple.json'  # 반드시 'region' 컬럼과 feature.properties.name이 일치해야 함

# ▶️ GeoJSON 로드 (시도+시군구 이름이 properties.name으로 되어 있어야 함)
with open(geo_path, 'r', encoding='utf-8') as f:
    geo = json.load(f)

# ▶️ 지도 시각화
m = folium.Map(location=[36.5, 127.5], zoom_start=8)

# 3. GeoJSON에서 각 구의 중심좌표(centroid) 추출
regions, lats, lngs = [], [], []
for feature in geo['features']:
    region_name = feature['properties']['name']
    geometry = feature['geometry']
    centroid = shape(geometry).centroid
    regions.append(region_name)
    lats.append(centroid.y)
    lngs.append(centroid.x)
geo_df = pd.DataFrame({'region': regions, 'lat': lats, 'lng': lngs})

# 4. 기존 데이터와 중심좌표 병합
df = pd.merge(df, geo_df, on='region', how='left')

# 스타일 함수 정의
def style_function(feature):
    region_name = feature['properties']['name']
    color = df.loc[df['region'] == region_name, 'color']
    if not color.empty:
        return {'fillColor': color.values[0], 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}
    else:
        return {'fillColor': 'gray', 'color': 'black', 'weight': 1, 'fillOpacity': 0.7}

# GeoJson 레이어 추가
folium.GeoJson(
    geo,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['name'])
).add_to(m)

# 마커 스타일 개선 (더 눈에 잘 보이도록)
for idx, row in df.iterrows():
    if not pd.isna(row['pet_play']):
        folium.Marker(
            location=[row['lat'], row['lng']],
            icon=folium.DivIcon(
                html=f"""<div style="
                    font-size: 13px; 
                    color: white; 
                    background: rgba(30, 144, 255, 0.8); 
                    border-radius: 15px; 
                    padding: 6px 10px;
                    font-weight: bold;
                    text-align: center;
                    border: 2px solid white;
                    box-shadow: 0 2px 4px rgba(0,0,0,0.3);
                    white-space: nowrap;
                    min-width: 220px;
                ">반려동물 산책,놀이 펫세권: {row['region']}</div>""",
                icon_size=(80, 30),
                icon_anchor=(40, 15)
            ),
            tooltip=f"{row['region']}"
        ).add_to(m)

# 8. 지도 저장 또는 출력
m.save('../../../RESULT/visualization/pet_play_folium_choropleth.html')
m